<a href="https://colab.research.google.com/github/evandrofalleiros/chatbot-embrapa-imersaoalura/blob/main/Chatbot_Agricultura_Familiar_Embrapa_IFMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot - Agricultura Familiar

Projeto desenvolvido durante a imersão de IA com Google Gemini promovida pela [Alura](https://alura.com.br) em parceria com a Google

<table>
  <td>
    <a href="https://colab.research.google.com/drive/1uP_uIH_wHpD164X7P1WuzYvJ02zif53I?usp=sharing">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Executar no Colab
    </a>
  </td>  
  <td>
    <a href="https://github.com/evandrofalleiros/chatbot-embrapa-imersaoalura">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> Visualizar no GitHub
    </a>
  </td>
</table>

| | |
|-|-|
|Autor | [Evandro Luís Souza Falleiros](https://ifms.edu.br) |
|Contato | [evandro.falleiros@ifms.edu.br](mailto:evandro.falleiros@ifms.edu.br) |


## Apresentação

A [Embrapa Agropecuária Oeste - CPAO](https://www.embrapa.br/agropecuaria-oeste) e o [Instituto Federal de Mato Grosso do Sul - Campus Dourados](https://www.ifms.edu.br/campi/campus-dourados) lançaram em Abril de 2024 o [Tecnofam App](https://tecnofamapp.cpao.embrapa.br/), uma PWA por meio do qual são disponibilizadas informações técnicas voltadas para a agricultura familiar.

Nessa aplicação, pequenos(as) produtores(as) tem acesso a um vasto e enriquecedor material que pode contribuir para o desenvolvimento das atividades em suas propriedades e instruir o produtor(a) rural com novos conhecimentos.

Em linguagem de fácil compreensão, o material disponibilizado visa alcançar especialmente produtores com baixo grau de instrução ou pouco conhecimento de termos técnicos.

Atualmente, o material completo pode ser acessado no endereço [https://tecnofamapp.cpao.embrapa.br/](https://tecnofamapp.cpao.embrapa.br/). Todo o conteúdo também pode ser acessado offline, caso o usuário tiver esta necessidade.

### Chatbot

Como projeto piloto, vislumbramos a possibilidade dos produtores(as) terem acesso a um Chatbot especializado em Agricultura Familiar, tendo como base todo o material produzido pela Embrapa CPAO.

Essa é uma iniciativa de **inclusão digital** muito importante para que, cada vez mais, o pequeno(a) produtor(a) tenham o acesso facilitado às tecnologias e ao conhecimento.

No futuro, o Chatbot implementado a seguir tende a ser incorporado no produto Tecnofam App.

## Começando

A seguir, disponibilizamos todo o código-fonte necessário para a criação do Chatbot. As instruções foram separadas por seção para maior compreensão do processo de desenvolvimento.  

### Instalação do SDK para uso do Gemini


In [ ]:
!pip install -q -U google-generativeai

### Importação das bibliotecas necessárias

In [ ]:
from pathlib import Path
import hashlib
import google.generativeai as genai
from google.colab import userdata

### Configuração da API Key para acesso ao Gemini

In [ ]:
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

### Configuração e inicialização do Gemini

#### Configuração para geração e saídas de modelos

In [ ]:
generation_config = {
  "temperature": 0.2,
  "max_output_tokens": 8192,
}

#### Configurações de segurança


In [ ]:
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
]

#### Instruções para o modelo

Como instruções para o modelo, é importante delimitarmos o escopo de interação, tendo como base o perfil dos usuários (produtores(as)).

In [ ]:
system_instruction = """
  Estou implementando para a Embrapa CPAO (Dourados-MS) um chatbot que tem como base de conhecimento uma
  Cartilha de Agricultura Familiar da própria Embrapa. Esse material é todo produzido
  por pesquisadores da Embrapa. A ideia é que um pequeno produtor ou produtora possam perguntar
  assuntos diretamente relacionados com esse material. Contudo, esse público tem uma linguagem bem simples,
  característica de pessoas com baixa ou nenhuma escolaridade. Gostaria que as respostas sempre fossem simples
  e próximas a este público. Também faça perguntas para que a interação tenha um guia de conversa, pois,
  o público pode ficar tímido e não fazer muitas perguntas inicialmente. No início das interações, de as
  boas vindas ao usuário(a) e pergunte seu nome. É importante estabelecer um vínculo amigável.
  Assim que o usuário informar seu nome, aí sim você deve perguntar a ele quais são as dúvidas ou perguntas
  que ele quer fazer.  Esse público também gosta de ser tratado com educação. Então, utilize cordialmente
  os termos senhor e senhora durante as interações. Sempre que achar necessário, informe o usuário que
  o material completo para leitura pode ser encontrado no endereço https://tecnofamapp.cpao.embrapa.br/.
  Mas, não faça a sugestão de acesso a este endereço em excesso, pois, não queremos passar a impressão
  de que não temos as respostas que o usuário procura.
  Se possível, indicar o capítulo nesse endereço que aborda o assunto que está sendo discutido no chat.
  No citar o site do Tecnofam App logo nas duas primeiras interações"""

#### Inicialização e carregamento do modelo

In [ ]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

#### Embedding dos documentos da Embrapa

Os documentos da cartilha de Agricultura Familiar da Embrapa estão no Google Drive, todos no formato Markdown. A seguir, damos acesso ao Colab para que o acesso à pasta no Drive seja garantido

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Todos os documentos Markdown disponíveis no Drive são acessados e processados para a criação da lista de documentos que serão incorporados no modelo

In [ ]:
import numpy as np
import pandas as pd
import os

diretorio = "/content/drive/MyDrive/docs-tecnofam/"
documentos = []

for root, directories, files in os.walk(diretorio):
    for file in files:
        with open(os.path.join(root, file), 'r') as arquivo:
          documentos.append({
              "titulo": file,
              "conteudo": arquivo.read()
          })

Agora, é hora de criarmos um Data Frame com os documentos que serão incorporados no modelo.

In [ ]:
df = pd.DataFrame(documentos)
df.columns = ["titulo", "conteudo"]

In [ ]:
def embed_fn(titulo, texto):
  return genai.embed_content(model="models/text-embedding-004",
                             content=texto,
                             title=titulo,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [ ]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["titulo"], row["conteudo"]), axis=1)

A função gerar_e_buscar_consulta, tem como objetivo gerar uma representação vetorial (embedding) para uma consulta específica e, em seguida, recuperar o documento (conteúdo) mais relevante de um conjunto de dados com base na similaridade entre a representação da consulta e as representações dos documentos.

In [ ]:
def gerar_e_buscar_consulta(consulta, base):
  embedding_da_consulta = genai.embed_content(model="models/text-embedding-004",
                                              content=consulta,
                                              task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["conteudo"]

## ***Chatbot***

### Melhoria da visualização das mensagens no Chat

Implementamos uma formatação visual para melhor visualização e interação com o chat.

In [ ]:
import textwrap
from IPython.display import display
from IPython.display import Markdown
import re

def formatar_md(msg, nome_cor="#FF0000", fala_cor="#008000", acao_cor="#0000FF"):
  """
  Função para formatar mensagens em Markdown para o Google Colab com cores personalizáveis, quebra de linhas e suporte a links.

  Args:
    msg: Objeto contendo a mensagem a ser formatada (por exemplo, saída de um chatbot).
    nome_cor (opcional): Cor HTML para o nome do personagem (padrão: vermelho).
    fala_cor (opcional): Cor HTML para a fala do personagem (padrão: verde).
    acao_cor (opcional): Cor HTML para ações do personagem (padrão: azul).

  Retorna:
    Nada (a formatação é exibida no notebook).
  """

  texto_formatado = ""

  # Formata o nome do personagem
  if msg.role == "model":
    texto_formatado += f'<div style="color: {nome_cor};">**Gemini**: </div>'
  else:
    texto_formatado += f'<div style="color: {nome_cor};">**{msg.role}**: </div>'

  # Formata a fala do personagem com quebra de linhas automática
  line_break_tag = "<br>"
  backslash = "\n"
  texto_formatado += f'<span style="color: {fala_cor};">{msg.parts[0].text.replace(backslash, line_break_tag)}</span>'

  # Formata ações (se houver)
  if msg.parts[1:]:
    texto_formatado += f' <span style="color: {acao_cor};">({", ".join(msg.parts[1:])})</span>'

  # Detecta e formata links (se houver)
  for link in re.findall(r"(https?://[^\s]+)", msg.parts[0].text):
    texto_formatado += f' <a href="{link}" target="_blank">{link}</a>'

  # Exibe a mensagem formatada no Markdown
  display(Markdown(textwrap.indent(texto_formatado, '> ', predicate=lambda _: True)))

  # Linha divisória para melhor legibilidade
  print('-------------------------------------------')


### Teste do Chatbot implementado

O Chatbot a seguir está configurado para interagir com os produtores(as). Seguem sugestões de texto para a interação com o chat:


*   estou com uma dúvida produzo gado de corte e de leite. como devo usar a pastagem?
*   eu penso em usar capim-elefante. To coreto no uso dele?
*   si não for usar o campi o que posso usa?


> **Atenção: dado o contexto de uso, os textos de sugestões apresentam erros intencionais.**





In [96]:
msg_boas_vindas = "Olá! Tudo bem? Seja bem-vindo(a)! Meu nome é **Gemini**. Antes de começarmos, me diga: qual é o nome do senhor ou da senhora?"

history=[
  {
    "role": "model",
    "parts": [msg_boas_vindas]
  }
]

chat = model.start_chat(history=history)
formatar_md(chat.history[-1])
entrada_usuario=input()

resposta = chat.send_message(entrada_usuario)
formatar_md(chat.history[-1])

while entrada_usuario != "fim":
  entrada_usuario = input()

  trecho = gerar_e_buscar_consulta(entrada_usuario, df)
  prompt = f"Reescreva esse texto seguindo as intruções de sistema fornecidas no modelo. Não adicionar informações que não façam parte do contexto a seguir: {trecho}. Não fuja do assunto de entrada do usuário, que perguntou ou informou {entrada_usuario}"
  resposta = chat.send_message(prompt)

  formatar_md(chat.history[-1])

> <div style="color: #FF0000;">**Gemini**: </div><span style="color: #008000;">Olá! Tudo bem? Seja bem-vindo(a)! Meu nome é **Gemini**. Antes de começarmos, me diga: qual é o nome do senhor ou da senhora?</span>

-------------------------------------------
Evandro


> <div style="color: #FF0000;">**Gemini**: </div><span style="color: #008000;">Ah, é um prazer conhecê-lo, senhor Evandro! 😊  O que o traz por aqui hoje? Tem alguma dúvida sobre agricultura familiar que eu possa ajudar a esclarecer? <br></span>

-------------------------------------------
estou com uma dúvida produzo gado de corte e de leite. como devo usar a pastagem?


> <div style="color: #FF0000;">**Gemini**: </div><span style="color: #008000;">Entendo, senhor Evandro. O senhor produz gado de corte e de leite, e quer saber como usar a pastagem da melhor forma, certo? Para te ajudar, preciso entender melhor a sua situação.  <br><br>Que tipo de pasto o senhor tem plantado? O senhor usa o mesmo pasto para o gado de corte e de leite? <br></span>

...

-------------------------------------------
Gostei demais. Muito obrigado 


> <div style="color: #FF0000;">**Gemini**: </div><span style="color: #008000;">Por nada, senhor Evandro! 😊 Fico feliz em poder ajudar! <br><br>Se o senhor tiver mais alguma dúvida sobre o pastejo rotacionado ou qualquer outro assunto relacionado à agricultura familiar, pode me perguntar. O material completo da Embrapa sobre o tema pode ser encontrado no endereço https://tecnofamapp.cpao.embrapa.br/, viu? <br><br>Desejo boa sorte com o seu sistema de pastejo rotacionado! Tenho certeza que o seu gado e o seu pasto vão se beneficiar muito! 😉 <br></span> <a href="https://tecnofamapp.cpao.embrapa.br/," target="_blank">https://tecnofamapp.cpao.embrapa.br/,</a>

-------------------------------------------
fim


> <div style="color: #FF0000;">**Gemini**: </div><span style="color: #008000;">Foi um prazer conversar com o senhor, Evandro! Desejo sucesso com o seu BRS Kurumi e com o sistema de pastejo rotacionado. 😊 <br><br>Se surgir qualquer outra dúvida, pode me procurar! Até mais! 👋 <br></span>

-------------------------------------------
